In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

system_messages = [
    "You are a helpful assistant",  # Default message
    "You are a friendly advisor",
    "You are a knowledgeable guide",
    "You are a witty companion"
]

# Internal model names with user-friendly display names
models = {
    "phi-3.1-mini-128k-instruct": "Phi (Microsoft)",
    "granite-3.0-2b-instruct": "Granite (IBM)",
    "gemma-2-2b-instruct": "Gemma (Google)",
    "llama-3.2-3b-instruct": "LLaMA (Meta)"
}


In [3]:
# Extracting internal model names and their corresponding display names
internal_models = list(models.keys())  # Internal names for API
display_models = list(models.values())  # Display names for the dropdown

def chat(message, selected_model_display, selected_system_message, history=None):
    # Initialize history if None
    if history is None:
        history = []

    # Map the display name back to the internal model name
    selected_model = [k for k, v in models.items() if v == selected_model_display][0]

    # Prepare the messages for the API call
    messages = [{"role": "system", "content": selected_system_message}]  # Use selected system message
    
    # Append all history messages in the correct format
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    # Append the current user message
    messages.append({"role": "user", "content": message})

    # Create a chat completion request
    try:
        response = client.chat.completions.create(model=selected_model, messages=messages)
        assistant_response = response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return "Sorry, I encountered an error. Please try again.", history

    # Update history with the new message and response
    history.append((message, assistant_response))  # Ensure history is updated correctly
    
    return history, history  # Return updated history for the chatbot


In [5]:
def clear_chat():
    """Clear the chat history."""
    return [], []  # Return empty history and state

# Set up the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("<h1 style='text-align: center;'>Chatbot Interface</h1>")
    gr.Markdown("<p style='text-align: center;'>Choose a model and start chatting with the assistant!</p>")
    
    with gr.Row():
        model_selection = gr.Dropdown(
            choices=display_models,  # Use the display names for the dropdown
            label="Select Model",
            value=display_models[0],  # Default selection
            interactive=True
        )
        
        system_message_selection = gr.Dropdown(
            choices=system_messages,  # System messages dropdown
            label="Select System Message",
            value=system_messages[0],  # Default selection
            interactive=True
        )

        clear_button = gr.Button("Clear")  # Button to clear the chat

    chatbot = gr.Chatbot()  # Chatbot for displaying history
    user_input = gr.Textbox(placeholder="Type your message here...", label="Your Message")  # Textbox for user input
    state = gr.State([])  # Maintain chat history as state

    # Define the function that gets triggered on user message submission
    user_input.submit(chat, inputs=[user_input, model_selection, system_message_selection, state], outputs=[chatbot, state])
    user_input.submit(lambda: "", outputs=user_input)  # Clear the input box after submission
    clear_button.click(clear_chat, outputs=[chatbot, state])  # Connect the clear button to clear_chat function

iface.launch(server_name="0.0.0.0", server_port=7899)


C:\Users\iei1\.conda\envs\llms\Lib\site-packages\gradio\components\chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7899

To create a public link, set `share=True` in `launch()`.
